# Classificação de Vinhos com KNN e K-Fold Cross-Validation
Neste notebook, utilizamos o classificador K-Nearest Neighbors (KNN) para classificar vinhos com base em suas características, utilizando o dataset da UCI. Avaliaremos o desempenho do modelo com diferentes valores de `random_state` e número de vizinhos `k` através de K-Fold Cross-Validation.

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


In [ ]:
# Leitura do dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

# Exibir uma amostra dos dados
vinhos.head()

In [ ]:
# Pré-processamento dos dados
X = vinhos.drop('classe', axis=1)
y = vinhos['classe']


In [ ]:
# Função para realizar K-Fold Cross-Validation com KNN
def knn_kfold_cross_val(X, y, k_values, random_state):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)
    results = {}
    for k in k_values:
        accuracies = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracies.append(accuracy)
        results[k] = {'mean_accuracy': sum(accuracies)/len(accuracies), 'std_accuracy': pd.Series(accuracies).std()}
    return results


In [ ]:
# Realizar os experimentos com diferentes random_states
random_states = [42, 17, 24]
k_values = [3, 5]
all_results = {}

for random_state in random_states:
    all_results[random_state] = knn_kfold_cross_val(X, y, k_values, random_state)

# Exibir os resultados dos experimentos
all_results

## Análise dos Resultados
Aqui estão os resultados da validação cruzada para diferentes valores de `k` (3 e 5) e `random_state` (42, 17 e 24). A seguir, será feita uma análise da acurácia média e do desvio padrão para cada configuração.


In [ ]:
# Organizar os resultados em DataFrame para melhor visualização
results_df = pd.DataFrame(columns=['random_state', 'k', 'mean_accuracy', 'std_accuracy'])
for random_state, res in all_results.items():
    for k, metrics in res.items():
        results_df = results_df.append({
            'random_state': random_state,
            'k': k,
            'mean_accuracy': metrics['mean_accuracy'],
            'std_accuracy': metrics['std_accuracy']
        }, ignore_index=True)

results_df

### Conclusão
Com base nos resultados da validação cruzada, podemos identificar o melhor valor de `k` para cada `random_state`. Caso haja empate nas acurácias médias, registraremos isso na análise. A escolha do melhor `k` deve ser feita considerando a acurácia média mais alta e o desvio padrão mais baixo, indicando um modelo mais robusto.